In [1]:
import pandas as pd
from pathlib import Path

df = pd.read_csv("campagne.csv")

df2 = df[df["verification"] == "v1"]
df2 = df2[df2["bathy (yes/no)"] == "yes"]

print(len(df), len(df2))

325 177


In [10]:
def get_path(row):
    name = row["session name"]

    date = name.split("_")[0][0:6]


    letter_disk = ""
    if date in ["202312", "202311"]:
        letter_disk = "D"
    elif date in ["202211", "202309", "202310"]:
        letter_disk = "E1"
    else:
        letter_disk = "F"

    if 1 <= int(date[5]) <= 7 and int(date[3]) == 3 and int(date[4]) == 0 and "MDG" not in name:
        date = "202301-07"

    return f"/media/bioeos/{letter_disk}/{date}_plancha_session"

df2["root_path"] = df2.apply(lambda row: get_path(row), axis=1)

df3 = df2[["root_path", "session name"]]
df3.to_csv("filter.csv", index=False)

In [12]:
from pathlib import Path
import shutil 
for i, row in df3.iterrows():

    if "2025" not in row["session name"]: continue
    bathy_raster = Path(row["root_path"], row["session name"], "PROCESSED_DATA", "BATHY", f"{row['session name']}_bathy_raster-linear.tif")


    if not bathy_raster.exists():
        print("Aled", bathy_raster)
    else:
        shutil.copy(bathy_raster, f"/home/bioeos/Documents/project_hub/tms-server/bathy/data/{bathy_raster.name}")

In [1]:
from pathlib import Path
import shutil

output = Path("/home/bioeos/Documents/project_hub/tms-server/bathy/data")
output.mkdir(exist_ok=True)
for subfolder in Path("/home/bioeos/Documents/project_hub/tms-server/bathy/BATHY_RASTER").iterdir():
    for file in subfolder.iterdir():
        shutil.copy(file, Path(output, file.name))